# Naive Bayes

## A simple example on a toy problem

### Exercise

Automatic text classification is one of the standard Machine Learning applications. Spam filtering was the first service based on Machine Learing used by almost every internet users.

We want to build a email classification system using a Naive Bayes classifier.
Emails must be classified into 2 different classes : *membership* or *complaint*.

 The classification will be based on the presence or absence of 3 words : 
 
 `join, address,  unacceptable, `
 
The training sample is composed of following documents, presented using a bag-of-word representation : 


join |  address | unacceptable  | class 
---|--------|-------|--------
1|1|0| membership
0|0|0| membership
1|1|0| membership
1|1|0| membership
1|1|0| membership
0|1|1| membership
1|1|0| membership
0|1|1| complaint
1|1|0| complaint
0|0|1| complaint
0|1|1| complaint
0|1|1| complaint
1|0|0| complaint





We denote by  $Y$ the document class and  $X=X_1,\cdots,X_3$ the bag of word features.

**Question**:
> * Compute the frequency tables for each event and each class
> 
> Class |  "join" present | "address" present  | "unacceptable" present 
> ---|--------|-------|--------
> membership | | |
> complaint | | |
>  
> * Divide the values in the table by the frequency of each class to compute P(word i is present|Y)
> * Compute the probability of P(word i is absent|Y) : this is 1-P(word i is present|Y)
> * Compute the prior  probabilities P(Y=membership) and P(Y=complaint)


We want to predict the class of the following document using a Naives Bayes classifier 

*Sir, I have a new problem with my account : I can not login. This is the third time this week that my account is down and I can join no one. This is unacceptable.*

In this text : "join" is present, "address" is not present and "unacceptable" is present

**Question**:
    
>Based on the probabilities computed before, compute the most probable class using the Bayes Formula :
>
> $$ P(Y | X_1,X_2,X_3) = \frac{P(X_1 | Y)P(X_2 | Y)P(X_3 | Y)P(Y)}{P(X)}$$

> To simplify the task, we have already computed  P(X) = 0.04203733
>
> You will have to compute  :
>
> * $P(Y = membership | X_1,X_2,X_3)$   = P("join" is present | Y = membership) \* P("address" is absent | Y = membership ) \* P("unacceptable" is present| Y = membership) \* P( Y = membership) / P(X)
> * $P(Y = complaint | X_1,X_2,X_3)$   = P("join" is present | Y = complaint) \* P("address" is absent | Y = complaint ) \* P("unacceptable" is present| Y = complaint) \* P( Y = complaint) / P(X)
> * and select the class with the maximal probability.
> 

In [1]:
import pandas as pd
import numpy as np
df = pd.DataFrame([
    [1,1,0, 'membership'],
[0,0,0, 'membership'],
[1,1,0, 'membership'],
[1,1,0, 'membership'],
[1,1,0, 'membership'],
[0,1,1, 'membership'],
[1,1,0, 'membership'],
[0,1,1, 'complaint'],
[1,1,0, 'complaint'],
[0,0,1, 'complaint'],
[0,1,1, 'complaint'],
[0,1,1, 'complaint'],
[1,0,0, 'complaint'],
    ],
    columns=['join',  'address',  'unacceptable', 'class'],
)
import numpy as np
# Compute the number of occurrences per event and per class
print (df.groupby('class').sum())
print ()

# Compute the number of occurrences per class
#print (df['class'].value_counts())
sum = np.flip(np.tile(df['class'].value_counts().values, (3,1)).transpose(),axis=0)
print (sum)
print ()

# Compute the probability  per event and per class
p1 = df.groupby('class').sum().values  / sum
print ("P(X=1|C)")
print (p1)
print()

# Compute the probability of the complement event
p0 = 1 - p1
print ("P(X=0|C)")
print (p0)
print ()

# Compute prior
prior = np.flip(df['class'].value_counts().values.reshape((2,1)),axis=0) / 13.
print ("Prior(C)")
print (prior)
print ()

# Compute P(X)
px = p1[0][0]*p0[0][1]*p1[0][2]*prior[0]+ p1[1][0]*p0[1][1]*p1[1][2]*prior[1]
print ("P(X)=",px)
print()

print ("Compute log proba")
# Compute the log proba

lp1 = np.log(p1)
print ("Log proba")
print (lp1)
print ()

lp0 = np.log(p0)
print ("Log proba of the complement event")
print (lp0)
print ()

# Compute the prior and log prior
prior = np.flip(df['class'].value_counts().values.reshape((2,1)),axis=0) / 13.
lprior = np.log(prior)
print ("Log prior")
print (lprior)



            join  address  unacceptable
class                                  
complaint      2        4             4
membership     5        6             1

[[6 6 6]
 [7 7 7]]

P(X=1|C)
[[0.33333333 0.66666667 0.66666667]
 [0.71428571 0.85714286 0.14285714]]

P(X=0|C)
[[0.66666667 0.33333333 0.33333333]
 [0.28571429 0.14285714 0.85714286]]

Prior(C)
[[0.46153846]
 [0.53846154]]

P(X)= [0.04203733]

Compute log proba
Log proba
[[-1.09861229 -0.40546511 -0.40546511]
 [-0.33647224 -0.15415068 -1.94591015]]

Log proba of the complement event
[[-0.40546511 -1.09861229 -1.09861229]
 [-1.25276297 -1.94591015 -0.15415068]]

Log prior
[[-0.77318989]
 [-0.61903921]]


In [2]:
# Compute the posterior for each class
classes_name = list(df.groupby('class').groups.keys())

for c in [0,1]:
    print (classes_name[c],end='=')
    print (p1[c][0]*p0[c][1]*p1[c][2]*prior[c]/px)


complaint=[0.81327801]
membership=[0.18672199]


### Code

The following DataFrame contains the data of the previous exercice. 

**Question**
> * Train a [Bernouilli Naive Bayes](http://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.BernoulliNB.html) on all the data.
> * Predict the class of the sample `sample = np.array([1,0,1])`, corresponding to the document of the previous excercise. 
> * Check your prediction with `clf.predict_proba([1,0,1])`
> * Check the different probabilities computed for the previous question :
>  * `clf.feature_count_` :  the count for each feature per class $C(X_i =1 \| Y=c)$
>  * `clf.feature_log_prob_` : the log probability for each feature per class $log(P(X_i =1 \| Y=c))$
>  * `clf.class_log_prior_` : the log prior probabilities for each class $log(P(Y=c))$
> * Check your prediction with `clf.predict_proba([1,0,1])` 



In [3]:
import pandas as pd
import numpy as np
df = pd.DataFrame([
    [1,1,0, 'membership'],
[0,0,0, 'membership'],
[1,1,0, 'membership'],
[1,1,0, 'membership'],
[1,1,0, 'membership'],
[0,1,1, 'membership'],
[1,1,0, 'membership'],
[0,1,1, 'complaint'],
[1,1,0, 'complaint'],
[0,0,1, 'complaint'],
[0,1,1, 'complaint'],
[0,1,1, 'complaint'],
[1,0,0, 'complaint'],
    ],
    columns=['join',  'address',  'unacceptable', 'class'],
)
df

,join,address,unacceptable,class
0,1,1,0,membership
1,0,0,0,membership
2,1,1,0,membership
3,1,1,0,membership
4,1,1,0,membership
5,0,1,1,membership
6,1,1,0,membership
7,0,1,1,complaint
8,1,1,0,complaint
9,0,0,1,complaint


In [4]:
# Check that we have the same result with BernoulliNB from sklearn
# Define the features and class
X = df[['join',  'address',  'unacceptable']]
y = df['class']
from sklearn.naive_bayes import BernoulliNB
clf = BernoulliNB(alpha=0) # alpha = 0 to disable smoothing to compare with the previous exercise
clf.fit(X,y) # Your code here
print ("frequence per event and per class")
print (clf.feature_count_)
print ("proba per event and per class")
print (np.exp(clf.feature_log_prob_))
print ("class prior")
print (np.exp(clf.class_log_prior_))


frequence per event and per class
[[2. 4. 4.]
 [5. 6. 1.]]
proba per event and per class
[[0.33333333 0.66666667 0.66666667]
 [0.71428571 0.85714286 0.14285714]]
class prior
[0.46153846 0.53846154]


C:\Users\trist_7liu08x\anaconda3\lib\site-packages\sklearn\naive_bayes.py:511: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  warnings.warn('alpha too small will result in numeric errors, '


In [5]:
# create the vector for the text to be classified 
# "join" is present, address is absent and unacceptable is present -> [1,0,1]
sample = np.array([1,0,1])
# predict the class
print (clf.predict(sample.reshape(1, -1))) # reshape to avoid a warning
print (clf.classes_)
# predict the probability for each class
print (clf.predict_proba(sample.reshape(1, -1))) # reshape to avoid a warning

['complaint']
['complaint' 'membership']
[[0.81327801 0.18672199]]


## LeMonde2003 Dataset

We will now apply classification algorithms to newspaper articles published in 2003 in *Le Monde*. The dataset can be dowloaded in  CSV format [here](https://kermorvant.github.io/csexed-ml/data/LeMonde2003_9classes.csv.gz).

These articles concern different subjects but we will consider only articles related to the following subjects : entreprises (ENT), international (INT), arts (ART), société (SOC), France (FRA), sports (SPO), livres (LIV), télévision (TEL) and the font page articles (UNE).


> * Load the CSV file `LeMonde2003_9classes.csv` containing the articles using [pd.read_csv](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) . 
> * Plot the frequency histogram of the categories using [`countplot`](https://seaborn.pydata.org/tutorial/categorical.html) : `sns.countplot(data=df,y='category')`



In [ ]:
import pandas as pd
import seaborn as sns
%matplotlib inline
import numpy as np
np.random.seed(42)
# load dataframe from CSV file
df = pd.read_csv('../data/LeMonde2003_9classes.csv.gz')
print (df.shape)
df.head()

In [ ]:
# Plot the statistics of category
sns.countplot(data=df,y='category')

In [ ]:
# Print examples of the articles
print ("Category:",df.iloc[100]['category'])
print (df.iloc[100]['text'])
print ()
print ("Category:",df.iloc[10000]['category'])
print (df.iloc[10000]['text'])
print ()
print ("Category:",df.iloc[5008]['category'])
print (df.iloc[5008]['text'])


## Bag-of-word representation

In order to apply machine learning algorithms to text, documents must be transformed into vectors. The most simple and standard way to transform a document into a vector is the *bag-of-word* encoding.

The idea is very simple : 

1. define the set of all the possible words that can appear in a document; denote its size by `max_features`.
2. for each document,  encode it with a vector of size `max_features`, with the value of the ith component of the vector equal to the number of time the ith word appears in the document.

See [the wikipedia article on Bag-of-word](https://en.wikipedia.org/wiki/Bag-of-words_model) for an example.

Scikit-learn proposes different methods to encode text into vectors : [CountVectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) and [TfidfTransformer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfTransformer.html).

The encoder must first be trained on the train set and applied to the different sets, for example with the 1000  words : 

	from sklearn.feature_extraction.text import CountVectorizer
	vectorizer = CountVectorizer(max_features=1000)
    vectorizer.fit(X_train)
    X_train_counts = vectorizer.transform(X_train)
    X_test_counts = vectorizer.transform(X_test)
        
**Question**:

> * Split the dataset LeMonde2003 into train/dev/test set. 
> * For each set, transform the text of the articles into vectors using the `CountVectorizer` with `max_features=1000` words.



In [ ]:
from sklearn.model_selection import train_test_split
# Split the dataset (X and y together)
df_train, df_test = train_test_split(df ,test_size=0.20, random_state=42)
df_train_hyper, df_dev = train_test_split(df_train ,test_size=0.25, random_state=42)

print ('train size',df_train_hyper.shape)
print ('dev size', df_dev.shape)
print ('test size', df_test.shape)

In [ ]:
# create features X and target y
X_train = df_train_hyper.text
X_dev = df_dev.text
X_test = df_test.text
y_train = df_train_hyper.category
y_dev = df_dev.category
y_test = df_test.category

X_train_dev = df_train.text
y_train_dev = df_train.category


In [ ]:
# train a Naive Bayes classifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
# create the vectorizer object
vectorizer = CountVectorizer(max_features=2000)
# fit on train data
vectorizer.fit(X_train)
# apply it on train and dev data
X_train_counts = vectorizer.transform(X_train)
X_dev_counts = vectorizer.transform(X_dev)


In [ ]:
vectorizer.get_params()

In [ ]:
# Print one sample text
print (X_train.iloc[10000])
# and its Count (sparse) representation 
print (X_train_counts[10000])
# Inverse vocabulary 
terms = np.array(list(vectorizer.vocabulary_.keys()))
indices = np.array(list(vectorizer.vocabulary_.values()))
inverse_vocabulary = terms[np.argsort(indices)]
most_frequent = X_train_counts[10000].argmax()
print ("most frequent word =",inverse_vocabulary[most_frequent])

In [ ]:
# define a mutlinomial Naive Bayes
clf = MultinomialNB()
# Train 
clf.fit(X_train_counts,y_train)
# Evaluate 
print ("Train accuracy",clf.score(X_train_counts,y_train))
print ("Dev accuracy",clf.score(X_dev_counts,y_dev))

## TF-IDF representation

The `CountVectorizer` encodes the text using the raw frequencies of the words. However, words that are very frequent and appear in all the documents will have a strong weight whereas they are not discriminative. The *Term-Frequency Inverse-Document-Frequency* weighting scheme take into accound the number of documents in which a given word occurs. A word that appear in many document will have less weight. See [the wikipedia page](https://en.wikipedia.org/wiki/Tf%E2%80%93idf) for more details.

With scikit-learn, the `TfidfTransformer` is applied after the `CountVectorizer` :

	from sklearn.feature_extraction.text import TfidfTransformer
	tf_transformer = TfidfTransformer().fit(X_train_counts)
 	X_train_tf = tf_transformer.transform(X_train_counts)
	X_test_tf = tf_transformer.transform(X_test_counts)
	
**Question**:

> * Use the TF-IDF representation to train a Multinomial Naive Bayes classifier. Report your best test error rate and the error rates for all the configurations tested.


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import accuracy_score
alpha = 0.001
max_words =2000
max_df = 1.0
min_df = 0.0
vectorizer = CountVectorizer(max_features=max_words,max_df=max_df,min_df=min_df)
vectorizer.fit(X_train)
X_train_counts = vectorizer.transform(X_train)
X_dev_counts = vectorizer.transform(X_dev)

tf_transformer = TfidfTransformer().fit(X_train_counts)
X_train_tf = tf_transformer.transform(X_train_counts)
X_dev_tf = tf_transformer.transform(X_dev_counts)
                 
clf = MultinomialNB(alpha=alpha)
clf.fit(X_train_tf, y_train)
predict_train = clf.predict(X_train_tf)
print ("Train score",accuracy_score(y_train,predict_train))
predict_dev = clf.predict(X_dev_tf)
print ("Dev score",accuracy_score(y_dev,predict_dev))

## Error analysis

The classification error rate give an evaluation of the performance for all the classes. But since the classes are not equally distributed, they may not be equally well modelized. In order to get a better idea of the performance of the classifier, detailed metrics must be used : 

* [metrics.classification_report](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html) provides a detailed analysis per class : the precision (amongst all the example classified as class X, how many are really from the classX) and the recall (amongst all the example that are from the class X, how many are classified as class X) and the F-Score which is as a weighted harmonic mean of the precision and recall.
* [metrics.confusion_matrix](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html) which give the confusions between the classes.

**Question**:

> * Report the `classification_report` for your  classifier. Which classes have the best scores ? Why ?
> * Report the `confusion_matrix` for your  classifier. Which classes are the most confused ? Why ?


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_dev,predict_dev))

In [ ]:
from sklearn.metrics import confusion_matrix
conf_mat = confusion_matrix(y_dev,predict_dev)
print (conf_mat)

In [ ]:
# Better display of the confusion matrix
import itertools
import numpy as np
import matplotlib.pyplot as plt
def plot_confusion_matrix(cm, classes,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    """
    print(cm)
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

  
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], 'd'),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
plot_confusion_matrix(conf_mat,classes=sorted(y_train.unique()))

## Hyperparameter optimization

The classification process has many parameters : alpha for the classifier, max_words, max_df, min_df, ngram orders for the Count of TfIDF transformer. These parameters can be optimized by a grid search using GridSearchCV.

In [ ]:
# Hyperameters optimization with GridSearchCV = parallel processing
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from pprint import pprint
from time import time
import logging
# Display progress logs on stdout
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s %(levelname)s %(message)s')


pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', MultinomialNB()),
])


parameters = {
    'vect__max_df': (0.5, 0.75, 1.0),
    'vect__max_features': (None, 5000, 10000, 50000),
    'vect__ngram_range': ((1, 1), (1, 2)),  # unigrams or bigrams
    'tfidf__use_idf': (True, False),
    'tfidf__norm': ('l1', 'l2'),
    'clf__alpha': (0.0001, 0.001,0.01,0.1)
}
if __name__ == "__main__":
    # multiprocessing requires the fork to happen in a __main__ protected
    # block

    # find the best parameters for both the feature extraction and the
    # classifier
    grid_search = GridSearchCV(pipeline, parameters, n_jobs=-1, verbose=2)

    print("Performing grid search...")
    print("pipeline:", [name for name, _ in pipeline.steps])
    print("parameters:")
    pprint(parameters)
    t0 = time()
    grid_search.fit(X_train_dev[:1000], y_train_dev[:1000])
    print("done in %0.3fs" % (time() - t0))
    print()

    print("Best score: %0.3f" % grid_search.best_score_)
    print("Best parameters set:")
    best_parameters = grid_search.best_estimator_.get_params()
    for param_name in sorted(parameters.keys()):
        print("\t%s: %r" % (param_name, best_parameters[param_name]))